In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib qt
import sys; sys.path.insert(0, '../')
import numpy as np
from matplotlib import pyplot as plt
from scipy.stats import pearsonr
import mne
from esinet import Simulation
from esinet.forward import get_info, create_forward_model
from esinet.util import unpack_fwd
pp = dict(surface='white', hemi='both')

# Get Forward Model

In [2]:
info = get_info(kind='biosemi32')
fwd = create_forward_model(info=info, sampling='ico3')

leadfield, pos = unpack_fwd(fwd)[1:3]
n_chans, n_dipoles = leadfield.shape

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    1.9s remaining:    1.9s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    2.0s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    2.0s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    0.1s remaining:    0.1s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.1s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    0.1s remaining:    0.1s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.2s finished


# Get sample data

In [93]:
settings = dict(number_of_sources=3, extents=(25, 40), duration_of_trial=1, target_snr=10)

sim = Simulation(fwd, info, settings).simulate(2)
stc = sim.source_data[0]
evoked = sim.eeg_data[0].average()

brain = stc.plot(**pp)
brain.add_text(0.1, 0.9, 'Ground Truth', 'title',
               font_size=14)

-- number of adjacent vertices : 1284
Simulating data based on sparse patches.


100%|██████████| 2/2 [00:00<00:00, 282.17it/s]


source data shape:  (1284, 1000) (1284, 1000)


100%|██████████| 2/2 [00:00<00:00, 17.24it/s]

Using control points [1.18668609e-08 2.04346565e-08 6.08158610e-08]


For automatic theme detection, "darkdetect" has to be installed! You can install it with `pip install darkdetect`
To use light mode, "qdarkstyle" has to be installed! You can install it with `pip install qdarkstyle`


# Regularization Optimizations

In [95]:
from invert.solvers.multiple_sparse_priors import SolverMultipleSparsePriors
from invert.solvers.loreta import SolverLORETA, SolverSLORETA, SolverELORETA
from invert.solvers.wrop import SolverBackusGilbert, SolverLAURA
from invert.solvers.smap import SolverSMAP
from invert.solvers.minimum_norm_estimates import SolverDynamicStatisticalParametricMapping, SolverWeightedMinimumNorm, SolverMinimumNorm

solver = SolverLAURA()


solver.make_inverse_operator(fwd, alpha='auto')
stc_hat = solver.apply_inverse_operator(evoked)
stc_hat.plot(**pp, brain_kwargs=dict(title=solver.name))

solver = SolverLORETA()

solver.make_inverse_operator(fwd, alpha='auto')
stc_hat = solver.apply_inverse_operator(evoked)
stc_hat.plot(**pp, brain_kwargs=dict(title=solver.name))

c:\Users\Lukas\Documents\projects\invert\dev\..\invert\solvers\wrop.py:119: RuntimeWarning: divide by zero encountered in power
  A = -d**-drop_off


Using control points [2.16276313e-09 2.71430102e-09 8.08474223e-09]
For automatic theme detection, "darkdetect" has to be installed! You can install it with `pip install darkdetect`
To use light mode, "qdarkstyle" has to be installed! You can install it with `pip install qdarkstyle`
filtered out idx 11
filtered out idx 11
Using control points [1.12443118e-08 1.33682760e-08 3.81063807e-08]
For automatic theme detection, "darkdetect" has to be installed! You can install it with `pip install darkdetect`
To use light mode, "qdarkstyle" has to be installed! You can install it with `pip install qdarkstyle`


In [92]:
from invert.solvers.multiple_sparse_priors import SolverMultipleSparsePriors
from invert.solvers.loreta import SolverLORETA, SolverSLORETA, SolverELORETA
from invert.solvers.wrop import SolverBackusGilbert, SolverLAURA
from invert.solvers.smap import SolverSMAP
from invert.solvers.minimum_norm_estimates import SolverDynamicStatisticalParametricMapping, SolverWeightedMinimumNorm, SolverMinimumNorm
solvers = [SolverMultipleSparsePriors, SolverLORETA, SolverSLORETA, SolverELORETA, SolverBackusGilbert, SolverLAURA, SolverSMAP, SolverDynamicStatisticalParametricMapping, SolverWeightedMinimumNorm, SolverMinimumNorm]

for solver in solvers:
    solver_ = solver()
    if solver_.name == "Multiple Sparse Priors":
        solver_.make_inverse_operator(fwd, evoked, alpha='auto')
    else:
        solver_.make_inverse_operator(fwd, alpha='auto')
    stc_hat = solver_.apply_inverse_operator(evoked)
    stc_hat.plot(**pp, brain_kwargs=dict(title=solver_.name))



Using 12 temporal mode(s)
Iteration 1. Free Energy Improvement: 10.61
Iteration 2. Free Energy Improvement: 8.17
Iteration 3. Free Energy Improvement: 145.50
Iteration 4. Free Energy Improvement: 108.53
Iteration 5. Free Energy Improvement: 361.28
Iteration 6. Free Energy Improvement: 13.43
Iteration 7. Free Energy Improvement: 4.58
Iteration 8. Free Energy Improvement: 4.93
Iteration 9. Free Energy Improvement: 4.30
Iteration 10. Free Energy Improvement: 3.83
Iteration 11. Free Energy Improvement: 25.28
Iteration 12. Free Energy Improvement: 3.68
Iteration 13. Free Energy Improvement: 3.30
Iteration 14. Free Energy Improvement: 5.70
Iteration 15. Free Energy Improvement: 2.81
Iteration 16. Free Energy Improvement: 2.87
Iteration 17. Free Energy Improvement: 3.04
Iteration 18. Free Energy Improvement: 3.31
Iteration 19. Free Energy Improvement: 3.68
Iteration 20. Free Energy Improvement: 3.23
Iteration 21. Free Energy Improvement: 2.93
Iteration 22. Free Energy Improvement: 5.57
Iterat

c:\Users\Lukas\Documents\projects\invert\dev\..\invert\solvers\wrop.py:119: RuntimeWarning: divide by zero encountered in power
  A = -d**-drop_off


idx = 10, r=18.73817422860385
<class 'numpy.ndarray'> (1284, 1000)
Using control points [3.98340706e-10 4.83675241e-10 1.36553032e-09]
For automatic theme detection, "darkdetect" has to be installed! You can install it with `pip install darkdetect`
To use light mode, "qdarkstyle" has to be installed! You can install it with `pip install qdarkstyle`
idx = 3, r=0.0533669923120631
<class 'numpy.ndarray'> (1284, 1000)
Using control points [5.89482238e-09 7.10940597e-09 1.87693903e-08]
For automatic theme detection, "darkdetect" has to be installed! You can install it with `pip install darkdetect`
To use light mode, "qdarkstyle" has to be installed! You can install it with `pip install qdarkstyle`
alpha must be set to a float when using Dynamic Statistical Parametric Mapping, auto does not work yet.
<class 'numpy.ndarray'> (1284, 1000)
Using control points [3.24547326e-06 4.08672745e-06 1.41410127e-05]
For automatic theme detection, "darkdetect" has to be installed! You can install it with 

In [17]:
from mne.minimum_norm import make_inverse_operator as mne_inverse
from mne.minimum_norm import apply_inverse as mne_apply
from mne import make_ad_hoc_cov
noise_cov = make_ad_hoc_cov(evoked.info, verbose=0)
mne_io = mne_inverse(evoked.info, fwd, noise_cov=noise_cov, fixed=True, loose=0, depth=0, verbose=0)
stc_hat = mne_apply(evoked, mne_io, method="MNE", verbose=0)
stc_hat.plot(**pp)

Using control points [4.75057550e-08 5.53502160e-08 1.36987867e-07]
For automatic theme detection, "darkdetect" has to be installed! You can install it with `pip install darkdetect`
To use light mode, "qdarkstyle" has to be installed! You can install it with `pip install qdarkstyle`
